In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


df = pd.read_csv("../car_prices.csv")

df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce', utc=True)
df['month'] = df['saledate'].dt.month
sales_data = df.groupby(['model', 'month']).agg({'sellingprice': 'count'}).reset_index()
sales_data.rename(columns={'sellingprice': 'sales_count'}, inplace=True)
df = pd.merge(sales_data, df, on=['model', 'month'], how='left')

df = df.drop(["vin", "mmr", "saledate", "trim"], axis = 1)

num = df.select_dtypes(include=[np.float64, np.int64]).columns.tolist()
categ = [col for col in df.columns if col not in num]

df = df.dropna()

print(df.shape, df.columns)

X = df.drop(['sales_count', "model", "month"], axis=1)
y = df['sales_count']
num.remove("sales_count")
categ.remove("model")
num.remove("month")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

X_train = pd.get_dummies(X_train, columns=categ, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categ, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

scaler = StandardScaler()
X_train[num] = pd.DataFrame(scaler.fit_transform(X_train[num]), index=X_train.index, columns=num)
X_test[num] = pd.DataFrame(scaler.transform(X_test[num]), index=X_test.index, columns=num)

C:\Users\davyd\AppData\Local\Temp\ipykernel_18444\2124280936.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['saledate'] = pd.to_datetime(df['saledate'], errors='coerce', utc=True)


(472325, 14) Index(['model', 'month', 'sales_count', 'year', 'make', 'body', 'transmission',
       'state', 'condition', 'odometer', 'color', 'interior', 'seller',
       'sellingprice'],
      dtype='object')


In [6]:
from keras.layers import Dropout
from keras.regularizers import l2

model_nn = Sequential([
    InputLayer((X_train.shape[1],)),
    Dense(128, activation="relu", kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(64, activation="relu", kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
    Dense(1)
])

model_nn.compile(optimizer=Adam(learning_rate = 0.001), loss='mean_squared_error')
model_nn.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 158s 11ms/step - loss: 801486.9375
Epoch 2/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 153s 11ms/step - loss: 613546.8125
Epoch 3/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 154s 11ms/step - loss: 569576.6875
Epoch 4/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 153s 11ms/step - loss: 547166.1875
Epoch 5/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 154s 11ms/step - loss: 531800.8125
Epoch 6/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 148s 11ms/step - loss: 517671.1875
Epoch 7/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 151s 11ms/step - loss: 512270.1250
Epoch 8/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 162s 12ms/step - loss: 510800.0312
Epoch 9/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 149s 11ms/step - loss: 501390.4688
Epoch 10/10
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 154s 11ms/step - loss: 498971.0625


In [7]:
y_pred = model_nn.predict(X_test)
y_pred_train = model_nn.predict(X_train)

r2_val = round(r2_score(y_test, y_pred), 4)
rmse_val = round(np.sqrt(mean_squared_error(y_test, y_pred)))

r2_train = round(r2_score(y_train, y_pred_train), 4)
rmse_train = round(np.sqrt(mean_squared_error(y_train, y_pred_train)))

print(r2_val, rmse_val)
print(r2_train, rmse_train)

739/739 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
14023/14023 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step
0.6427 705
0.6712 671


In [8]:
model_nn.save('model1.keras')

In [1]:
pip install modeling

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement modeling (from versions: none)
ERROR: No matching distribution found for modeling
